In [22]:
import os
import json
import pandas as pd
import traceback

In [23]:
import PyPDF2
# import torch
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
from langchain import HuggingFaceHub, LLMChain, PromptTemplate, HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM



In [ ]:
Key=os.getenv("OPENAI_API_KEY")

In [25]:
from dotenv import main

main.load_dotenv() # take environment variables from .env

True

In [26]:
# model_id="nvidia/Llama-3.1-Nemotron-70B-Instruct-HF"
# model_id="meta-llama/Llama-3.2-3B-Instruct"
model_id="google/flan-t5-large"
# model_id="meta-llama/Meta-Llama-3-8B-Instruct"

llm=HuggingFaceHub

In [27]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer"
    }
}

In [28]:
TEMPLATE="""
Text{text}
You are a expert MCQ maker. Given the above test, it is your job to \
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the question are not repeated and checks all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below adn use it as guide. \
Ensure to make {number} MCQS
*** RESPONSE_JSON ***
{response_json}
"""

In [29]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text', 'number', 'subject', "tone", 'response_json'],
    template=TEMPLATE
)

In [30]:
quiz_chain = LLMChain(llm=llm(repo_id=model_id, model_kwargs={'temperature':0.1}), prompt=quiz_generation_prompt, output_key='quiz', verbose=True)

In [31]:
TEMPLATE2="""
You are an expert english gramarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the questions and give a complete analysis of the quiz. Only use at max 50 words for complexity if the quiz is not at per with the congitive and analytical abilities of the students.\
Update the quiz questions which needs to be changed the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [32]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE2
)

In [33]:
review_chain = LLMChain(llm=llm(repo_id=model_id, model_kwargs={'temperature':0.5}), prompt=quiz_evaluation_prompt, output_key='review', verbose=True)

In [34]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)


In [35]:
file_path=r"../data.txt"

In [36]:
file_path


'../data.txt'

In [37]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [38]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Quick progress in the field of deep learning, beginning in 2010s, allowed neural networks to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoretical viewpoint, probably approximately correct (PAC) learning provides a framework for describing machine lea

In [39]:
# Serialize  the Python dictionary into a JSON-formatted string

json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [40]:
NUMBER = 5,
SUBJECT = "Machine Learning",
TONE = "Simple"


In [41]:
# For OpenAI  
# How to setup Token Usage  Tracking  in Langchain

with get_openai_callback() as cb:
    response=generate_evaluate_chain(
                {
                    "text": TEXT,
                    "number": NUMBER,
                    "subject": SUBJECT, 
                    "tone": TONE,
                    "response_json": RESPONSE_JSON
                }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

TextMachine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Quick progress in the field of deep learning, beginning in 2010s, allowed neural networks to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]

From a theoretic

In [42]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0


In [43]:
print(response)

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Quick progress in the field of deep learning, beginning in 2010s, allowed neural networks to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.\n\nStatistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]\n\nFrom a theoretical viewpoint, probably approximately correct (PAC) learning provides a framework for descri

In [44]:
quiz=response.get("quiz")


In [45]:
print(quiz)


'1': 'mcq': 'multiple


In [46]:
quiz=json.loads(quiz)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [47]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

AttributeError: 'str' object has no attribute 'items'

In [48]:
quiz=pd.DataFrame(quiz_table_data)

In [49]:
quiz.to_csv("Machine_Learning.csv", index=False)

In [50]:
from datetime import datetime
# datetime.now().strftime("%d/%m/%Y, %H:%M:%S")
datetime.now().strftime("%d_%m_%Y, %H_%M_%S")

'10_11_2024, 00_19_12'

In [51]:
print( f"{datetime.now().strftime('%d_%m_%Y, %H_%M_%S')}.log")

10_11_2024, 00_19_15.log
